Practicing using BERT through movie review sentiment classification.

Author: Jerry Zou

Because MacBERTh is built on BERT, there only needs minimum change to the code in this file when we transition to MacBERTh.

In [ ]:
print("Importing libraries...")
import pandas as pd
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
import pyarrow as pa
from datasets import Dataset

print("Libraries uploaded.")
dataFrame = pd.read_csv("IMDBDataset.csv")
dataFrame.head()
print("Importing tokenizer...")
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
print("Tokenizer imported.")

def processData(row):
    text = row["review"]
    text = str(text)
    text = " ".join(text.split())
    encoder = tokenizer(text, padding="max_length", truncation=True, max_length=128)
    label = 0
    if row["sentiment"] == "positive":
        label +=1
    
    encoder["label"] = label
    encoder["text"] = text
    return encoder

print("Processing...")
print(processData({
    "review": "this is a sample review of a movie.",
    "sentiment": "positive"
}))

processedData = []
for i in range(len(dataFrame[:100])):
    processedData.append(processData(dataFrame.iloc[i])) 

from sklearn.model_selection import train_test_split

new_df = pd.DataFrame(processedData)

train_df, valid_df = train_test_split(
    new_df,
    test_size=0.2,
    random_state=2022
)

train_hg = Dataset(pa.Table.from_pandas(train_df))
valid_hg = Dataset(pa.Table.from_pandas(valid_df))
print(type(train_hg))
print(type(valid_hg))

model = AutoModelForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels=2
)

training_args = TrainingArguments(output_dir="./result", evaluation_strategy="epoch")

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_hg,
    eval_dataset=valid_hg,
    tokenizer=tokenizer
)
print(type(train_hg))
trainer.train()
trainer.evaluate()
model.save_pretrained('./modelTrained/')

In [2]:
#load the model
newModel = AutoModelForSequenceClassification.from_pretrained("modelTrained")

Importing libraries...


2024-05-27 15:58:33.851891: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Libraries uploaded.
Importing tokenizer...
Tokenizer imported.
Processing...
{'input_ids': [101, 2023, 2003, 1037, 7099, 3319, 1997, 1037, 3185, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


<class 'datasets.arrow_dataset.Dataset'>


In [3]:
newTokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

NameError: name 'AutoTokenizer' is not defined

In [ ]:
import torch
import numpy as mp
def getPrediction(text):
    encoding = newTokenizer(text, return_tensors="pt", padding="max_length", truncation=True, max_length=128)
    encoding = {k: v.to(textclassificationpractice.trainer.model.device) for k, v in encoding.items()}
    outputs = newModel(**encoding)
    logits = outputs.logits
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(logits.squeeze().cpu())
    probs = probs.detach().numpy()
    label = mp.argmax(probs, axis=-1)

    if label == 1:
        return{
            "sentiment": "Positive",
            "probability": probs[1]
        }
    else:
        return{
            "sentiment": "Negative",
            "probability": probs[0]
        }

In [ ]:
getPrediction("I love today's weather.")

NameError: name 'getPrediction' is not defined